## Resultados do BLAST

A partir dos resultados do blast obtidos (o top 10 é apresentado nas tabelas abaixo) para cada uma das nossas proteínas, é possível inferir a função de algumas por homologia. Contudo, a maior parte dos resultados com melhores valores de percentagem de identidade, *score* e *e-value* são entradas *unreviewed*, o que explica o facto de muitas das entradas não terem função associada. Esta inferência da função é especialmente importante no caso das proteínas hipotéticas, ou outras proteínas menos estudadas. Para uma outra conseguimos, portanto, inferir a sua função, como é o caso, por exemplo, da proteína hipotética com a locus tag __lpg0235__, a qual é possível que tenha a atividade de liase carbono-enxofre. Não podemos, no entanto, afirmar com 100% de certeza que esta proteína desempenha tal função, uma vez que, não só os resultados do BLAST são *unreviewed*, mas também porque tal facto ainda não foi comprovado experimentalmente. Para as proteínas mais estudadas, tanto neste organismo como noutros que possuem proteínas homólogas, os resultados do BLAST têm, mais frequentemente, as funções moleculares das respetivas proteínas descritas, sendo que as mesmas são iguais ao longo da tabela respetiva. 


Para além da função, consideramos pertinente retirar também o organismo a que cada proteína que obtivemos como resultado do BLAST pertence, por forma a auxiliar a construção posterior de árvores filogenéticas e, também, no mesmo sentido, percebermos em que organismos é que as proteínas em estudo possuem homólogas. Isto torna-se relevante, pois se verificamos que o organismo é mais próximo daquele em estudo, por exemplo uma susbspécie ou estirpe diferente da mesma bactéria, a inferência da função é muito mais forte. 


A inferência da função foi feita de três maneiras diferentes, duas automáticas e uma manual. Quanto à inferência automática, as funções prováveis foram inferidas, verificando quais as que aparecem com maior frequência em relação ao número de entradas. Isto foi feito tanto para o top 10 como para o total de entradas do blast (no máximo 50). A inferência manual foi realizada tendo atenção, da mesma forma, à frequência da função, mas também do organismo. 

In [1]:
import os, sys, inspect
import pandas as pd
from IPython.core.display import display, HTML

def import_modules():
    """
    Importar os módulos que desenvolvemos neste trabalho.
    """
    current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
    parent_dir = os.path.dirname(current_dir)
    sys.path.insert(0, parent_dir)
    
def bold_it(text, bold):
    """
    Torna o texto bold se "bold" é True.
    """
    if bold:
        return "<strong>" + text + "</strong>"
    else:
        return text

def link_it(href, text, link):
    """
    Criar um link se "link" é True,
    """
    if link:
        return "<a href=\"#" + href + "\">" + text + "</a>"
    else:
        return text
    
def itemize(l, style="", bold=False, link=False):
    """
    Criar uma lista HTML dada uma lista.
    """
    html = "<div style=\"" + style + "\"><ul>"
    for i in l:
        html += "<li>"
        html += link_it(
            i,
            bold_it(i, bold),
            link
        )
        html += "</li>"
    html += "</ul></div>"
    return html
        

def main():
    import_modules()
    import util.rw as rw

    # não truncar informação
    pd.set_option('display.max_colwidth', -1)

    blast_results = rw.read_json("files/.blast_results.json")
    ncbi = rw.read_json("files/.ncbi.json")
    inferred = rw.read_json("files/.inferred.json")
    manual = rw.read_json("files/cur_manual.json")
    
    tags = sorted(blast_results.keys())
    
    # Mostrar lista de tags com links para os resultados
    css = "column-count:6; -webkit-column-count:6; -moz-column-count:6"
    html = itemize(tags, css, bold=True, link=True)
    display(HTML(html))
  
    for tag in tags:
        # Título antes de cada tabela
        product = ncbi[tag]["product"]
        display(HTML("<a name=\"" + tag + "\"></a><h2>" + tag + " " + product + "</h2>"))
        
        # Mostrar apenas o top 10
        top_ten = blast_results[tag][0:10]
        columns_to_show = ["uniprot_id", "identity", "score", "evalue", "status", "organism", "molecular_functions"]
        
        # Mostrar tabela
        df = pd.DataFrame.from_dict(top_ten)
        df["identity"] = df["identity"].apply(lambda i : str(round(i, 1)) + "%")
        df["molecular_functions"] = df["molecular_functions"].apply(itemize)
        display(HTML(df[columns_to_show].to_html(escape=False)))

        # Mostrar funções mais prováveis
        inferred_top_ten = inferred[tag]["top_ten"]
        inferred_all = inferred[tag]["all"]
        if len(inferred_top_ten) > 0:
            display(HTML("<h4>Função provável (Top 10):</h4>"))
            display(HTML(itemize(inferred_top_ten)))

        # Se o top 10 for diferente do top all, mostrar o top all
        if (not inferred_top_ten == inferred_all) and len(inferred_all) > 0:
            display(HTML("<h4>Função provável:</h4>"))
            display(HTML(itemize(inferred_all)))
        
        # Mostrar função proveniente do NCBI
        if "function" in ncbi[tag]:
            display(HTML("<h4>Função NCBI:</h4>"))
            display(HTML(itemize([ncbi[tag]["function"]])))
        
        # Mostrar curação manual
        if  tag in manual:
            display(HTML("<h4>Função Provável (Curação Manual) :</h4>"))
            display(HTML(itemize(manual[tag])))
            
main()

,uniprot_id,identity,score,evalue,status,organism,molecular_functions
0,G8US81,100.0%,949.0,2.496650e-128,unreviewed,Legionella pneumophila subsp. pneumophila ATCC 43290,"DNA bindingtranscription factor activity, sequence-specific DNA binding"
1,Q5ZYX9,100.0%,948.0,3.311010e-128,unreviewed,Legionella pneumophila subsp. pneumophila (strain Philadelphia 1 / ATCC 33152 / DSM 7513),"DNA bindingtranscription factor activity, sequence-specific DNA binding"
2,A0A129BU56,98.9%,938.0,9.820510e-127,unreviewed,Legionella pneumophila,"DNA bindingtranscription factor activity, sequence-specific DNA binding"
3,A0A182DGJ8,100.0%,795.0,2.038700e-105,unreviewed,Legionella pneumophila,"DNA bindingtranscription factor activity, sequence-specific DNA binding"
4,Q5WZU6,99.3%,792.0,6.419330e-105,unreviewed,Legionella pneumophila (strain Lens),"DNA bindingtranscription factor activity, sequence-specific DNA binding"
5,A0A193HI95,99.3%,792.0,6.419330e-105,unreviewed,Legionella pneumophila,
6,A0A128LCR8,98.6%,789.0,1.753620e-104,unreviewed,Legionella pneumophila,"DNA bindingtranscription factor activity, sequence-specific DNA binding"
7,A0A131BMT3,98.6%,788.0,2.774460e-104,unreviewed,Legionella pneumophila,"DNA bindingtranscription factor activity, sequence-specific DNA binding"
8,A0A140J9F8,95.3%,767.0,4.366440e-101,unreviewed,Legionella pneumophila subsp. pascullei,"DNA bindingtranscription factor activity, sequence-specific DNA binding"
9,A0A0W0VPN0,95.3%,764.0,1.205550e-100,unreviewed,Legionella lansingensis,"DNA bindingtranscription factor activity, sequence-specific DNA binding"


,uniprot_id,identity,score,evalue,status,organism,molecular_functions
0,Q5ZYX8,100.0%,2757.0,0.0,unreviewed,Legionella pneumophila subsp. pneumophila (strain Philadelphia 1 / ATCC 33152 / DSM 7513),benzoylformate decarboxylase activitymagnesium ion bindingthiamine pyrophosphate binding
1,G8US82,100.0%,2757.0,0.0,unreviewed,Legionella pneumophila subsp. pneumophila ATCC 43290,catalytic activitymagnesium ion bindingthiamine pyrophosphate binding
2,Q6BBR4,100.0%,2757.0,0.0,unreviewed,Legionella pneumophila,catalytic activitymagnesium ion bindingthiamine pyrophosphate binding
3,A0A182DKE6,99.1%,2742.0,0.0,unreviewed,Legionella pneumophila,benzoylformate decarboxylase activitythiamine pyrophosphate binding
4,A0A140J0X1,98.9%,2737.0,0.0,unreviewed,Legionella pneumophila subsp. pascullei,catalytic activitymagnesium ion bindingthiamine pyrophosphate binding
5,A0A130GQE1,98.3%,2725.0,0.0,unreviewed,Legionella pneumophila,benzoylformate decarboxylase activitymagnesium ion bindingthiamine pyrophosphate binding
6,A0A129HDX2,98.1%,2716.0,0.0,unreviewed,Legionella pneumophila,benzoylformate decarboxylase activitythiamine pyrophosphate binding
7,A0A131L8L7,97.9%,2707.0,0.0,unreviewed,Legionella pneumophila,benzoylformate decarboxylase activitythiamine pyrophosphate binding
8,A0A128RCH3,90.8%,2541.0,0.0,unreviewed,Legionella pneumophila,catalytic activitymagnesium ion bindingthiamine pyrophosphate binding
9,A0A0W0XJI7,76.7%,2152.0,0.0,unreviewed,Legionella quateirensis,catalytic activitythiamine pyrophosphate binding


,uniprot_id,identity,score,evalue,status,organism,molecular_functions
0,Q5ZYX7,100.0%,7772.0,0.0,unreviewed,Legionella pneumophila subsp. pneumophila (strain Philadelphia 1 / ATCC 33152 / DSM 7513),
1,G8US83,100.0%,7772.0,0.0,unreviewed,Legionella pneumophila subsp. pneumophila ATCC 43290,
2,Q6BBR6,100.0%,7772.0,0.0,unreviewed,Legionella pneumophila,
3,Q6RCR1,100.0%,7663.0,0.0,unreviewed,Legionella pneumophila,
4,Q5WZU3,98.1%,7552.0,0.0,unreviewed,Legionella pneumophila (strain Lens),
5,A0A182DDD3,97.9%,7522.0,0.0,unreviewed,Legionella pneumophila,
6,A0A193HAM8,96.7%,7459.0,0.0,unreviewed,Legionella pneumophila,
7,A0A140J9G0,97.2%,7458.0,0.0,unreviewed,Legionella pneumophila subsp. pascullei,
8,A0A128J5J3,96.6%,7455.0,0.0,unreviewed,Legionella pneumophila,
9,A0A128LC83,96.9%,7438.0,0.0,unreviewed,Legionella pneumophila,


,uniprot_id,identity,score,evalue,status,organism,molecular_functions
0,Q5ZYX6,100.0%,898.0,6.326980e-121,unreviewed,Legionella pneumophila subsp. pneumophila (strain Philadelphia 1 / ATCC 33152 / DSM 7513),carbon-sulfur lyase activity
1,G8US84,99.4%,897.0,2.169920e-120,unreviewed,Legionella pneumophila subsp. pneumophila ATCC 43290,carbon-sulfur lyase activity
2,Q6BBR7,100.0%,862.0,1.515370e-115,unreviewed,Legionella pneumophila,carbon-sulfur lyase activity
3,A0A129GSB4,98.1%,852.0,5.835440e-114,unreviewed,Legionella pneumophila,carbon-sulfur lyase activity
4,A0A193HAI4,97.5%,848.0,1.740850e-113,unreviewed,Legionella pneumophila,
5,Q5WZU2,97.5%,846.0,3.951540e-113,unreviewed,Legionella pneumophila (strain Lens),carbon-sulfur lyase activity
6,A0A182DEW6,97.5%,846.0,3.951540e-113,unreviewed,Legionella pneumophila,carbon-sulfur lyase activity
7,A0A129BTX4,97.5%,844.0,7.448560e-113,unreviewed,Legionella pneumophila,carbon-sulfur lyase activity
8,A0A130H3N2,95.6%,835.0,2.134090e-111,unreviewed,Legionella pneumophila,carbon-sulfur lyase activity
9,A0A128WCT2,93.8%,822.0,2.033460e-109,unreviewed,Legionella pneumophila,carbon-sulfur lyase activity


,uniprot_id,identity,score,evalue,status,organism,molecular_functions
0,Q5ZYX5,100.0%,3464.0,0.0,unreviewed,Legionella pneumophila subsp. pneumophila (strain Philadelphia 1 / ATCC 33152 / DSM 7513),
1,G8US85,100.0%,3464.0,0.0,unreviewed,Legionella pneumophila subsp. pneumophila ATCC 43290,
2,A0A182DIW0,97.7%,3399.0,0.0,unreviewed,Legionella pneumophila,
3,A0A131HDR5,97.6%,3385.0,0.0,unreviewed,Legionella pneumophila,
4,Q5WZU1,97.4%,3384.0,0.0,unreviewed,Legionella pneumophila (strain Lens),
5,A0A130KW89,97.3%,3384.0,0.0,unreviewed,Legionella pneumophila,
6,A0A128VZN5,97.0%,3363.0,0.0,unreviewed,Legionella pneumophila,
7,A0A129BU24,96.8%,3359.0,0.0,unreviewed,Legionella pneumophila,
8,A0A140ISL5,95.6%,3333.0,0.0,unreviewed,Legionella pneumophila subsp. pascullei,
9,A0A128RCH4,92.9%,3240.0,0.0,unreviewed,Legionella pneumophila,


...

,uniprot_id,identity,score,evalue,status,organism,molecular_functions
0,Q5ZYB3,100.0%,1381.0,0.000000e+00,unreviewed,Legionella pneumophila subsp. pneumophila (strain Philadelphia 1 / ATCC 33152 / DSM 7513),
1,G8URH7,100.0%,1381.0,0.000000e+00,unreviewed,Legionella pneumophila subsp. pneumophila ATCC 43290,
2,Q5WZ81,100.0%,1381.0,0.000000e+00,unreviewed,Legionella pneumophila (strain Lens),
3,Q7WUK3,100.0%,1381.0,0.000000e+00,unreviewed,Legionella pneumophila,
4,A0A140JA15,97.7%,1354.0,0.000000e+00,unreviewed,Legionella pneumophila subsp. pascullei,
5,A0A128PJU7,97.3%,1351.0,0.000000e+00,unreviewed,Legionella pneumophila,
6,A0A0A2T5B8,90.0%,1204.0,2.711970e-164,unreviewed,Legionella norrlandica,
7,A0A0W0YEQ3,85.4%,1159.0,1.686270e-157,unreviewed,Legionella quateirensis,
8,A0A0W0WK15,83.5%,1132.0,1.948290e-153,unreviewed,Legionella moravica,
9,A0A0W1AFP0,82.8%,1124.0,3.230340e-152,unreviewed,Legionella worsleiensis,


,uniprot_id,identity,score,evalue,status,organism,molecular_functions
0,Q5ZYB2,100.0%,2736.0,0.0,unreviewed,Legionella pneumophila subsp. pneumophila (strain Philadelphia 1 / ATCC 33152 / DSM 7513),IMP cyclohydrolase activityphosphoribosylaminoimidazolecarboxamide formyltransferase activity
1,G8URH8,100.0%,2736.0,0.0,unreviewed,Legionella pneumophila subsp. pneumophila ATCC 43290,IMP cyclohydrolase activityphosphoribosylaminoimidazolecarboxamide formyltransferase activity
2,A0A131MPU3,98.7%,2707.0,0.0,unreviewed,Legionella pneumophila,IMP cyclohydrolase activityphosphoribosylaminoimidazolecarboxamide formyltransferase activity
3,A0A182DHM6,98.7%,2706.0,0.0,unreviewed,Legionella pneumophila,IMP cyclohydrolase activityphosphoribosylaminoimidazolecarboxamide formyltransferase activity
4,A0A131MHJ7,98.5%,2704.0,0.0,unreviewed,Legionella pneumophila,IMP cyclohydrolase activityphosphoribosylaminoimidazolecarboxamide formyltransferase activity
5,A0A128R6N0,98.3%,2693.0,0.0,unreviewed,Legionella pneumophila,IMP cyclohydrolase activityphosphoribosylaminoimidazolecarboxamide formyltransferase activity
6,Q5WZ80,97.9%,2689.0,0.0,unreviewed,Legionella pneumophila (strain Lens),IMP cyclohydrolase activityphosphoribosylaminoimidazolecarboxamide formyltransferase activity
7,A0A129A9L0,97.4%,2669.0,0.0,unreviewed,Legionella pneumophila,IMP cyclohydrolase activityphosphoribosylaminoimidazolecarboxamide formyltransferase activity
8,A0A128LZX9,94.3%,2608.0,0.0,unreviewed,Legionella pneumophila,IMP cyclohydrolase activityphosphoribosylaminoimidazolecarboxamide formyltransferase activity
9,A0A140IT70,93.8%,2604.0,0.0,unreviewed,Legionella pneumophila subsp. pascullei,IMP cyclohydrolase activityphosphoribosylaminoimidazolecarboxamide formyltransferase activity


,uniprot_id,identity,score,evalue,status,organism,molecular_functions
0,G8URH9,100.0%,1395.0,0.0,unreviewed,Legionella pneumophila subsp. pneumophila ATCC 43290,protein methyltransferase activity
1,Q5ZYB1,100.0%,1395.0,0.0,reviewed,Legionella pneumophila subsp. pneumophila (strain Philadelphia 1 / ATCC 33152 / DSM 7513),protein methyltransferase activity
2,A0A182DQ73,99.7%,1387.0,0.0,unreviewed,Legionella pneumophila,protein methyltransferase activity
3,Q5WZ79,99.7%,1387.0,0.0,reviewed,Legionella pneumophila (strain Lens),protein methyltransferase activity
4,A0A130JLA9,99.3%,1385.0,0.0,unreviewed,Legionella pneumophila,protein methyltransferase activity
5,A0A129B3I7,98.3%,1374.0,0.0,unreviewed,Legionella pneumophila,protein methyltransferase activity
6,Q5X7S8,98.3%,1374.0,0.0,reviewed,Legionella pneumophila (strain Paris),protein methyltransferase activity
7,A0A128N0X7,97.9%,1369.0,0.0,unreviewed,Legionella pneumophila,protein methyltransferase activity
8,A0A193HB76,97.6%,1367.0,0.0,unreviewed,Legionella pneumophila,methyltransferase activity
9,A0A128PJM8,97.6%,1362.0,0.0,unreviewed,Legionella pneumophila,protein methyltransferase activity


,uniprot_id,identity,score,evalue,status,organism,molecular_functions
0,Q5ZYB0,100.0%,2270.0,0.0,unreviewed,Legionella pneumophila subsp. pneumophila (strain Philadelphia 1 / ATCC 33152 / DSM 7513),ATP bindingbiotin carboxylase activitymetal ion binding
1,G8URI0,100.0%,2270.0,0.0,unreviewed,Legionella pneumophila subsp. pneumophila ATCC 43290,ATP bindingbiotin carboxylase activitymetal ion binding
2,Q5WZ78,100.0%,2257.0,0.0,unreviewed,Legionella pneumophila (strain Lens),ATP bindingbiotin carboxylase activitymetal ion binding
3,A0A131MN61,100.0%,2257.0,0.0,unreviewed,Legionella pneumophila,ATP bindingbiotin carboxylase activitymetal ion binding
4,A0A129A9H2,99.8%,2255.0,0.0,unreviewed,Legionella pneumophila,ATP bindingbiotin carboxylase activitymetal ion binding
5,A0A130CY36,99.8%,2253.0,0.0,unreviewed,Legionella pneumophila,ATP bindingbiotin carboxylase activitymetal ion binding
6,A0A140IT72,99.8%,2252.0,0.0,unreviewed,Legionella pneumophila subsp. pascullei,ATP bindingbiotin carboxylase activitymetal ion binding
7,A0A128PJV5,99.8%,2252.0,0.0,unreviewed,Legionella pneumophila,acetyl-CoA carboxylase activityATP bindingbiotin carboxylase activitymetal ion binding
8,A0A0W0YET9,93.7%,2148.0,0.0,unreviewed,Legionella quateirensis,ATP bindingbiotin carboxylase activitymetal ion binding
9,A0A0W1A2N0,93.5%,2142.0,0.0,unreviewed,Legionella waltersii,ATP bindingbiotin carboxylase activitymetal ion binding


,uniprot_id,identity,score,evalue,status,organism,molecular_functions
0,A0A140J158,100.0%,474.0,5.852640e-59,unreviewed,Legionella pneumophila subsp. pascullei,rRNA bindingstructural constituent of ribosome
1,A0A0C9NJ73,100.0%,474.0,5.852640e-59,unreviewed,Legionella pneumophila,rRNA bindingstructural constituent of ribosome
2,Q5WZK8,100.0%,474.0,5.852640e-59,reviewed,Legionella pneumophila (strain Lens),rRNA bindingstructural constituent of ribosome
3,A5IHR0,100.0%,474.0,5.852640e-59,reviewed,Legionella pneumophila (strain Corby),rRNA bindingstructural constituent of ribosome
4,Q5X855,100.0%,474.0,5.852640e-59,reviewed,Legionella pneumophila (strain Paris),rRNA bindingstructural constituent of ribosome
5,A0A0W0WKD3,98.9%,473.0,8.028630e-59,unreviewed,Legionella moravica,rRNA bindingstructural constituent of ribosome
6,A0A0W0YED4,98.9%,473.0,8.028630e-59,unreviewed,Legionella quateirensis,rRNA bindingstructural constituent of ribosome
7,A0A0W0YW01,98.9%,473.0,8.028630e-59,unreviewed,Legionella shakespearei DSM 23087,rRNA bindingstructural constituent of ribosome
8,A0A0A2SNW0,98.9%,473.0,9.051400e-59,unreviewed,Legionella norrlandica,rRNA bindingstructural constituent of ribosome
9,A0A078L212,97.8%,471.0,1.527370e-58,unreviewed,Legionella massiliensis,rRNA bindingstructural constituent of ribosome
